# Лабораторная работа №2

## Реализация глубокой нейронной сети

В работе предлагается использовать набор данных _notMNIST_, который состоит из изображений размерностью 28×28 первых 10 букв латинского алфавита (_A_ ... _J_, соответственно). Обучающая выборка содержит порядка 500 тыс. изображений, а тестовая – около 19 тыс.

Данные можно скачать по ссылке:

* https://commondatastorage.googleapis.com/books1000/notMNIST_large.tar.gz (большой набор данных);

* https://commondatastorage.googleapis.com/books1000/notMNIST_small.tar.gz (маленький набор данных);

Описание данных на английском языке доступно по ссылке:
http://yaroslavvb.blogspot.sg/2011/09/notmnist-dataset.html

### Задание 1

Реализуйте полносвязную нейронную сеть с помощью библиотеки _TensorFlow_. В качестве алгоритма оптимизации можно использовать, например, стохастический градиент (_Stochastic Gradient Descent_, _SGD_). Определите количество скрытых слоев от 1 до 5, количество нейронов в каждом из слоев до нескольких сотен, а также их функции активации (кусочно-линейная, сигмоидная, гиперболический тангенс и т.д.).

In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [0]:
BASE_DIR = '/content/drive/My Drive/Colab Files/mo-2'

import sys

sys.path.append(BASE_DIR)

import os

os.chdir(BASE_DIR)

In [0]:
import pandas as pd

dataframe = pd.read_pickle("./large.pkl")

In [4]:
dataframe['data'].shape

(461946,)

In [5]:
! pip install tensorflow-gpu --pre --quiet

! pip show tensorflow-gpu

Name: tensorflow-gpu
Version: 2.2.0rc2
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: astunparse, google-pasta, six, scipy, termcolor, opt-einsum, keras-preprocessing, tensorflow-estimator, gast, numpy, protobuf, grpcio, wrapt, wheel, tensorboard, h5py, absl-py
Required-by: 


In [0]:
import tensorflow as tf

In [0]:
import numpy as np

In [8]:
x = np.asarray(list(dataframe['data']))[..., np.newaxis]

x = tf.keras.utils.normalize(x, axis = 1)

x.shape

(461946, 28, 28, 1)

In [0]:
IMAGE_DIM_0, IMAGE_DIM_1 = x.shape[1], x.shape[2]

In [10]:
from tensorflow.keras.utils import to_categorical

y = to_categorical(dataframe['label'].astype('category').cat.codes.astype('int32'))

y.shape

(461946, 10)

In [0]:
LAYER_WIDTH = 5000

In [0]:
CLASSES_N = y.shape[1]

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape

model = tf.keras.Sequential()

model.add(Reshape((IMAGE_DIM_0 * IMAGE_DIM_1,), input_shape = (IMAGE_DIM_0, IMAGE_DIM_1, 1)))
model.add(Dense(LAYER_WIDTH, activation = 'relu'))
model.add(Dense(LAYER_WIDTH, activation = 'sigmoid'))
model.add(Dense(LAYER_WIDTH, activation = 'tanh'))
model.add(Dense(LAYER_WIDTH, activation = 'elu'))
model.add(Dense(LAYER_WIDTH, activation = 'softmax'))
model.add(Dense(CLASSES_N))

In [0]:
def cat_cross_from_logits(y_true, y_pred):
    return tf.keras.losses.categorical_crossentropy(y_true, y_pred, from_logits = True)

model.compile(optimizer = 'sgd',
              loss = cat_cross_from_logits,
              metrics = ['categorical_accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 5000)              3925000   
_________________________________________________________________
dense_1 (Dense)              (None, 5000)              25005000  
_________________________________________________________________
dense_2 (Dense)              (None, 5000)              25005000  
_________________________________________________________________
dense_3 (Dense)              (None, 5000)              25005000  
_________________________________________________________________
dense_4 (Dense)              (None, 5000)              25005000  
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5

In [0]:
BATCH_SIZE = 128

In [0]:
r = 3608

In [0]:
VAL_SPLIT_RATE = 0.1

In [0]:
EPOCHS_N = 20

In [20]:
model.fit(x = x[:r * BATCH_SIZE], y = y[:r * BATCH_SIZE], epochs = EPOCHS_N, batch_size = BATCH_SIZE,
          validation_split = VAL_SPLIT_RATE)

Epoch 1/20
3248/3248 [==============================] - 62s 19ms/step - loss: 2.2421 - categorical_accuracy: 0.1129 - val_loss: 3.7853 - val_categorical_accuracy: 0.0000e+00
Epoch 2/20
3248/3248 [==============================] - 61s 19ms/step - loss: 2.2144 - categorical_accuracy: 0.1133 - val_loss: 4.3620 - val_categorical_accuracy: 0.0000e+00
Epoch 3/20
3248/3248 [==============================] - 61s 19ms/step - loss: 2.2083 - categorical_accuracy: 0.1127 - val_loss: 4.7218 - val_categorical_accuracy: 0.0000e+00
Epoch 4/20
3248/3248 [==============================] - 61s 19ms/step - loss: 2.2056 - categorical_accuracy: 0.1127 - val_loss: 4.9821 - val_categorical_accuracy: 0.0000e+00
Epoch 5/20
3248/3248 [==============================] - 61s 19ms/step - loss: 2.2042 - categorical_accuracy: 0.1132 - val_loss: 5.1854 - val_categorical_accuracy: 0.0000e+00
Epoch 6/20
3248/3248 [==============================] - 61s 19ms/step - loss: 2.2032 - categorical_accuracy: 0.1128 - val_loss: 5.

### Задание 2

Как улучшилась точность классификатора по сравнению с логистической регрессией?

### Задание 3

Используйте регуляризацию и метод сброса нейронов (_dropout_) для борьбы с переобучением. Как улучшилось качество классификации?

In [0]:
REG_RATE = 0.001

In [0]:
from tensorflow.keras.regularizers import l2

l2_reg = l2(REG_RATE)

In [0]:
DROPOUT_RATE = 0.2

In [0]:
from tensorflow.keras.layers import Dropout

dropout_layer = Dropout(DROPOUT_RATE)

In [0]:
model_2 = tf.keras.Sequential()

model_2.add(Reshape((IMAGE_DIM_0 * IMAGE_DIM_1,), input_shape = (IMAGE_DIM_0, IMAGE_DIM_1, 1)))
model_2.add(Dense(LAYER_WIDTH, activation = 'relu', kernel_regularizer = l2_reg))
model_2.add(dropout_layer)
model_2.add(Dense(LAYER_WIDTH, activation = 'sigmoid', kernel_regularizer = l2_reg))
model_2.add(dropout_layer)
model_2.add(Dense(LAYER_WIDTH, activation = 'tanh', kernel_regularizer = l2_reg))
model_2.add(dropout_layer)
model_2.add(Dense(LAYER_WIDTH, activation = 'sigmoid', kernel_regularizer = l2_reg))
model_2.add(dropout_layer)
model_2.add(Dense(LAYER_WIDTH, activation = 'relu', kernel_regularizer = l2_reg))
model_2.add(dropout_layer)
model_2.add(Dense(CLASSES_N))

In [0]:
model_2.compile(optimizer = 'sgd',
                loss = cat_cross_from_logits,
                metrics = ['categorical_accuracy'])

In [27]:
model_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 5000)              3925000   
_________________________________________________________________
dropout (Dropout)            (None, 5000)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 5000)              25005000  
_________________________________________________________________
dense_8 (Dense)              (None, 5000)              25005000  
_________________________________________________________________
dense_9 (Dense)              (None, 5000)              25005000  
_________________________________________________________________
dense_10 (Dense)             (None, 5000)             

In [28]:
model_2.fit(x = x[:r * BATCH_SIZE], y = y[:r * BATCH_SIZE], epochs = EPOCHS_N, batch_size = BATCH_SIZE,
            validation_split = VAL_SPLIT_RATE)

Epoch 1/20
3248/3248 [==============================] - 91s 28ms/step - loss: 22.2429 - categorical_accuracy: 0.1171 - val_loss: 24.7438 - val_categorical_accuracy: 0.0000e+00
Epoch 2/20
3248/3248 [==============================] - 91s 28ms/step - loss: 19.7819 - categorical_accuracy: 0.1298 - val_loss: 23.7590 - val_categorical_accuracy: 0.0000e+00
Epoch 3/20
3248/3248 [==============================] - 91s 28ms/step - loss: 17.2906 - categorical_accuracy: 0.2769 - val_loss: 21.7852 - val_categorical_accuracy: 0.0000e+00
Epoch 4/20
3248/3248 [==============================] - 91s 28ms/step - loss: 15.1711 - categorical_accuracy: 0.3934 - val_loss: 19.9977 - val_categorical_accuracy: 0.0000e+00
Epoch 5/20
3248/3248 [==============================] - 91s 28ms/step - loss: 13.3202 - categorical_accuracy: 0.5224 - val_loss: 18.5776 - val_categorical_accuracy: 0.0000e+00
Epoch 6/20
3248/3248 [==============================] - 91s 28ms/step - loss: 11.6135 - categorical_accuracy: 0.6610 - v

### Задание 4

Воспользуйтесь динамически изменяемой скоростью обучения (_learning rate_). Наилучшая точность, достигнутая с помощью данной модели составляет 97.1%. Какую точность демонстрирует Ваша реализованная модель?

In [29]:
from tensorflow.keras.optimizers import SGD

dyn_lr_sgd = SGD(lr = 0.01, momentum = 0.9)

model_2.compile(optimizer = dyn_lr_sgd,
                loss = cat_cross_from_logits,
                metrics = ['categorical_accuracy'])

model_2.fit(x = x[:r * BATCH_SIZE], y = y[:r * BATCH_SIZE], epochs = EPOCHS_N, batch_size = BATCH_SIZE,
            validation_split = VAL_SPLIT_RATE)

Epoch 1/20
3248/3248 [==============================] - 99s 30ms/step - loss: 1.7904 - categorical_accuracy: 0.7848 - val_loss: 6.7390 - val_categorical_accuracy: 0.0000e+00
Epoch 2/20
3248/3248 [==============================] - 99s 30ms/step - loss: 1.1303 - categorical_accuracy: 0.7947 - val_loss: 7.1027 - val_categorical_accuracy: 0.0000e+00
Epoch 3/20
3248/3248 [==============================] - 98s 30ms/step - loss: 0.9437 - categorical_accuracy: 0.7997 - val_loss: 6.2875 - val_categorical_accuracy: 0.0000e+00
Epoch 4/20
3248/3248 [==============================] - 99s 30ms/step - loss: 0.8940 - categorical_accuracy: 0.8028 - val_loss: 6.0434 - val_categorical_accuracy: 0.0000e+00
Epoch 5/20
3248/3248 [==============================] - 99s 30ms/step - loss: 0.8779 - categorical_accuracy: 0.8048 - val_loss: 6.2465 - val_categorical_accuracy: 0.0000e+00
Epoch 6/20
3248/3248 [==============================] - 99s 30ms/step - loss: 0.8706 - categorical_accuracy: 0.8071 - val_loss: 6.